In [15]:
from itertools import product

# Helper functions for unification and substitutions
def unify(x, y, theta={}):
    if theta is None:
        return None
    elif x == y:
        return theta
    elif isinstance(x, str) and x.islower():
        return unify_var(x, y, theta)
    elif isinstance(y, str) and y.islower():
        return unify_var(y, x, theta)
    elif isinstance(x, tuple) and isinstance(y, tuple) and len(x) == len(y):
        for xi, yi in zip(x, y):
            theta = unify(xi, yi, theta)
            if theta is None:
                return None
        return theta
    else:
        return None

def unify_var(var, x, theta):
    if var in theta:
        return unify(theta[var], x, theta)
    elif x in theta:
        return unify(var, theta[x], theta)
    else:
        theta[var] = x
        return theta

def substitute(clause, theta):
    def sub_literal(literal):
        predicate, terms = literal
        # Ensure terms are tuples, not lists
        return (predicate, tuple(theta.get(term, term) for term in terms))
    return {sub_literal(literal) for literal in clause}

def negate_literal(literal):
    # Negate a given literal
    predicate, terms = literal
    return (f"~{predicate}" if not predicate.startswith("~") else predicate[1:], terms)

# Convert KB and query into CNF
def to_cnf():
    KB = [
        {("mother", ("Leela", "Oshin"))},  # Leela is the mother of Oshin
        {("alive", ("Leela",))},  # Leela is alive
        {("mother", ("x", "y")), ("parent", ("x", "y"))},  # If x is mother of y, x is parent of y
        {("parent", ("x", "y")), ("alive", ("x", "y")), ("older", ("x", "y"))},  # If parent(x, y) and alive(x, y), then older(x, y)
    ]
    query = {("~older", ("Leela", "Oshin"))}  # Query: Prove that Leela is older than Oshin
    return KB + [query]

# Resolution procedure
def resolution(kb):
    clauses = kb
    new = set()
    while True:
        print("\nCurrent set of clauses:")
        for clause in clauses:
            print(clause)  # Debugging line to see the clauses in each iteration

        pairs = [(clauses[i], clauses[j]) for i in range(len(clauses)) for j in range(i + 1, len(clauses))]
        for (ci, cj) in pairs:
            resolvent = resolve(ci, cj)
            if resolvent is not None:
                if not resolvent:  # Empty clause found
                    print("Query proven by resolution!")  # Debugging line
                    return True
                new.add(frozenset(resolvent))
                print(f"New clauses after resolution: {new}")  # Debugging line
        if new.issubset(map(frozenset, clauses)):
            print("Query cannot be proven.")  # Debugging line
            return False
        clauses.extend(map(set, new))

def resolve(ci, cj):
    # Try to resolve two clauses ci and cj
    for di in ci:
        for dj in cj:
            print(f"Trying to resolve {di} with {dj}")  # Debugging line
            theta = unify(di, negate_literal(dj))
            if theta is not None:
                print(f"Unification successful with theta: {theta}")  # Debugging line
                resolvent = (ci - {di}) | (cj - {dj})
                resolvent = substitute(resolvent, theta)
                print(f"Resolvent: {resolvent}")  # Debugging line
                return resolvent
    return None

# Main function
def main():
    # Get the CNF of the knowledge base (KB)
    kb = to_cnf()
    # Perform resolution to prove the query
    result = resolution(kb)
    if result:
        print("Leela is older than Oshin is TRUE.")
    else:
        print("Leela is older than Oshin is FALSE.")

if __name__ == "__main__":
    main()



Current set of clauses:
{('mother', ('Leela', 'Oshin'))}
{('alive', ('Leela',))}
{('mother', ('x', 'y')), ('parent', ('x', 'y'))}
{('older', ('x', 'y')), ('alive', ('x', 'y')), ('parent', ('x', 'y'))}
{('~older', ('Leela', 'Oshin'))}
Trying to resolve ('mother', ('Leela', 'Oshin')) with ('alive', ('Leela',))
Trying to resolve ('mother', ('Leela', 'Oshin')) with ('mother', ('x', 'y'))
Unification successful with theta: {'mother': '~alive', '~alive': '~mother', 'x': 'Leela', 'y': 'Oshin'}
Resolvent: {('parent', ('Leela', 'Oshin'))}
New clauses after resolution: {frozenset({('parent', ('Leela', 'Oshin'))})}
Trying to resolve ('mother', ('Leela', 'Oshin')) with ('older', ('x', 'y'))
Unification successful with theta: {'mother': '~alive', '~alive': '~mother', 'x': 'Leela', 'y': 'Oshin', '~mother': '~older'}
Resolvent: {('parent', ('Leela', 'Oshin')), ('alive', ('Leela', 'Oshin'))}
New clauses after resolution: {frozenset({('parent', ('Leela', 'Oshin'))}), frozenset({('parent', ('Leela', 'O